In [1]:
import os
import g_requests
import parsers
os.getcwd()
import pandas as pd 

# Performing joint reconstructions.

Step 1) Submit the job to the server

In [2]:
g_requests.requestJointReconstruction(aln="../test_data/big_test_data/GRASPTutorial_Final.aln",
                                      nwk="../test_data/big_test_data/GRASPTutorial_Final.nwk")

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



'{"Message":"Queued","Job":22}\n'

Step 2) Find out where your job is in the queue

In [3]:
g_requests.requestPlaceInQueue(22)

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



'{"Job":22,"Place":0}\n'

Step 3) Retrieve your job which will have the POG graphs 

In [2]:
graphs = g_requests.requestJobOutput(22)


Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



Step 4 - Optional) Request POGs for extant sequences 

In [ ]:
extant_tree = g_requests.requestExtantPOGTree(aln="../test_data/big_test_data/GRASPTutorial_Final.aln",
nwk="../test_data/big_test_data/GRASPTutorial_Final.nwk")


Step 5 - Option 1) Build a POG tree from extant and ancesor POGs using both of the server outputs 

- The advantage of doing it this way is that the POGTree object will contain sequence information on the BranchPoints for ancestors AND extants 

In [7]:
tree = parsers.POGTreeFromJSON(extant_tree, graphs)

tree.branchpoints["XP_012687241.1"].seq

Step 5 - Option 2) Build a POGTree from the ancestor POG and from a nwk file string 

- ONLY ancestors will have sequence information based on the most likely symbol at each position in the sequence

In [3]:
tree = parsers.POGTreeFromJSON(nwk="../test_data/big_test_data/GRASPTutorial_Final.nwk", POG_graphs=graphs)

# Learning distributions from data 

The following instructions demonstrate how to learn a probability distribution from data. I need to add option to change some of the parameters as currently just runs on default settings. 

Step 1 - Optional) The data below is a demonstration of how data should be formated in a CSV.

- Currently only implemented to accept 1 data point per sequence

In [3]:
df = pd.DataFrame()

headers = ["A5ILB0","P08144","P29957","H2N0D4","T1WDH2","T1WE96","H9B4I9","A0A060DAC6","Q47R94","Q5UZY3","D8J7H2","Q4A3E0","L8B068","Q2QC88","O93647","Q4J9M2","Q97YY0","O33476","D0KTV8","P96107","P06279","P04746","P00690","O08452","D4P4Y7","Q1KLC8","Q8A1G3","P17654","B8Y698","P00692","B8Y1H0","B1VK33","B5ARZ9","P20845","B6RB08","P06278","P04063","P00693","Q8LJQ6","Q2KJQ1","Q6WUB6","A8VWC5","A0SEG1","P04745","Q75UG5"]
data = [[8.5],[7.35],[7.35],[7],[9],[9],[None],[8.5],[7.35],[None],[None],[None],[7],[5.75],[5.25],[3.25],[None],[6.5],[None],[None],[None],[None],[7.35],[5.75],[5.5],[7],[None],[None],[8],[6],[None],[7],[6.5],[None],[6.5],[6.466666666666666],[None],[None],[None],[None],[9.5],[None],[None],[6.9],[7]]

df["Headers"] = headers
df["Data"] = data

df.to_csv('train_data.csv', index=False)


Step 2) Send request to the server

In [2]:
g_requests.requestTrainFromData(nwk="./3_2_1_1_filt.nwk", states=["A", "B"], data="train_data.csv")

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



'{"Message":"Queued","Job":26}\n'

Step 3) Check the status of your job 

In [3]:
g_requests.requestPlaceInQueue(26)

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



'{"Job":26,"Place":0}\n'

Step 4) Retrieve your job and save the output

In [4]:
out = g_requests.requestJobOutput(26)

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



In [5]:
out

{'Job': 26,
 'Result': {'Distrib': {'Condition': [['A'], ['B']],
   'Pr': [[6.578658560392298, 0.5003342721876141],
    [8.632049675842909, 0.5003342721876141]],
   'Index': [0, 1],
   'Domain': 'dat.Continuous@6d74d0bf'}}}